In [1]:
#import packages

import os
from dotenv import load_dotenv

from langchain.document_loaders import YoutubeLoader
import youtube_transcript_api
import pytube


from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

In [2]:
#loading environment variables for api keys from .env file

%load_ext dotenv
%dotenv
chatGPT_api_key = os.getenv("CHATGPT_API_KEY")

## Youtube Transcripts Loading

In [6]:
#create a interfact allow for user to input youtube url

url = input("Enter a youtube url: ")

In [7]:
#loading transcripts from youtube videos by using url

#url = 'https://www.youtube.com/watch?v=OushE7mq0Ak' #url of the video (Apple Business Essentials)
loader = YoutubeLoader.from_youtube_url(
    url,
    add_video_info=True
)
loader.load()

[Document(page_content="♪ Thrilling cinematic music ♪ [REVVED ENGINE PASSING] [REVVING] [TIRES SQUEALING] [REVVING] ♪ [REVVING AND SCREECHING] [EXPLOSIONS] The titanium iPhone 15 Pro, built with\nan aerospace-grade material, a new level\nof mobile gaming power, and a camera that can literally\nshoot a blockbuster film. - Wait, what? A titanium gaming powerhouse\nmovie camera phone? Is that even a thing? - Yes, it is definitely a thing. Titanium, the same material\nused in spacecraft design, makes iPhone 15 Pro\nand iPhone 15 Pro Max our lightest Pro models ever. Titanium's unique properties\ngive it one of the highest strength-to-weight ratios\nof any metal. - Which basically\nmeans it's light enough to get up into space\nand strong enough to make it all the way\nback to Earth. Oops. - The new design\ncomes in four Pro finishes and now includes\na USB-C connector with superfast transfer speeds. And there's\na customizable Action button for quicker access\nto your favorite feature, from

In [8]:
#define video meta data
video_metadata  = loader.load()[0].metadata
video_title = video_metadata['title']
print(video_title)

Introducing iPhone 15 Pro | Apple


In [9]:
#define video transcript
transcript = loader.load()[0].page_content
print(transcript)

♪ Thrilling cinematic music ♪ [REVVED ENGINE PASSING] [REVVING] [TIRES SQUEALING] [REVVING] ♪ [REVVING AND SCREECHING] [EXPLOSIONS] The titanium iPhone 15 Pro, built with
an aerospace-grade material, a new level
of mobile gaming power, and a camera that can literally
shoot a blockbuster film. - Wait, what? A titanium gaming powerhouse
movie camera phone? Is that even a thing? - Yes, it is definitely a thing. Titanium, the same material
used in spacecraft design, makes iPhone 15 Pro
and iPhone 15 Pro Max our lightest Pro models ever. Titanium's unique properties
give it one of the highest strength-to-weight ratios
of any metal. - Which basically
means it's light enough to get up into space
and strong enough to make it all the way
back to Earth. Oops. - The new design
comes in four Pro finishes and now includes
a USB-C connector with superfast transfer speeds. And there's
a customizable Action button for quicker access
to your favorite feature, from opening the camera
to activating a sho

In [10]:
#saving the transcript in a text file with the name of the video automatically for future use in a transcript folder

# Remove invalid characters from title for filenames
valid_filename = "".join(c for c in video_title if c not in ('?', '/', '\\', ':', '*', '<', '>', '|', '"'))

# Set the directory name and filename
directory_name = "transcripts"
filename = f"{valid_filename}.txt"

# Ensure the directory exists
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Write the transcript to the file
with open(os.path.join(directory_name, filename), 'w', encoding="utf-8") as file:
    file.write(transcript)

print(f"Transcript saved to: {directory_name}/{filename}")

Transcript saved to: transcripts/Introducing iPhone 15 Pro  Apple.txt


In [11]:
# read all text files from a folder
directory = 'transcripts'
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding="utf-8") as file:
            text = file.read()
            print(text)

♪ Thrilling cinematic music ♪ [REVVED ENGINE PASSING] [REVVING] [TIRES SQUEALING] [REVVING] ♪ [REVVING AND SCREECHING] [EXPLOSIONS] The titanium iPhone 15 Pro, built with
an aerospace-grade material, a new level
of mobile gaming power, and a camera that can literally
shoot a blockbuster film. - Wait, what? A titanium gaming powerhouse
movie camera phone? Is that even a thing? - Yes, it is definitely a thing. Titanium, the same material
used in spacecraft design, makes iPhone 15 Pro
and iPhone 15 Pro Max our lightest Pro models ever. Titanium's unique properties
give it one of the highest strength-to-weight ratios
of any metal. - Which basically
means it's light enough to get up into space
and strong enough to make it all the way
back to Earth. Oops. - The new design
comes in four Pro finishes and now includes
a USB-C connector with superfast transfer speeds. And there's
a customizable Action button for quicker access
to your favorite feature, from opening the camera
to activating a sho

## Summarization

In [12]:
# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [13]:
# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k", openai_api_key= chatGPT_api_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

docs = loader.load()
summary_information = stuff_chain.run(docs)
print(summary_information)

In [14]:
summary_information = 'sumaary of the video for dmeo purpose'

In [15]:
# saved summary information in a text file with the name of the video automatically for future use in a summary folder

# Remove invalid characters from title for filenames
valid_filename = "".join(c for c in video_title if c not in ('?', '/', '\\', ':', '*', '<', '>', '|', '"'))

# Set the directory name and filename
directory_name = "summary"
filename = f"{valid_filename}.txt"

# Ensure the directory exists
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Write the transcript to the file
with open(os.path.join(directory_name, filename), 'w', encoding="utf-8") as file:
    file.write(summary_information)

print(f"Summary saved to: {directory_name}/{filename}")

Summary saved to: summary/Introducing iPhone 15 Pro  Apple.txt
